# imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *

import os
from Utils import cacheOnDisk
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done
done
done
done
stop


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate


'11072018'

# generate common sharing matrix

# generate merged phenotype table:

*generate merged phenotype table with indication for disease yes/no
*generate targets dataframe
*loop over 

## load seperate phenotype files:

In [ ]:
f1='%s/TCR_real_data/NewPhenotypicData/PNP530_AgeGenderBMIcreatSmoking.xlsx' %MyPath
PNP530_AgeGenderBMIcreatSmoking=pd.read_excel(f1).set_index('BD')
print len(PNP530_AgeGenderBMIcreatSmoking)
PNP530_AgeGenderBMIcreatSmoking.head()

In [ ]:
f2='%s/TCR_real_data/CardioSamples/phenotypicData/Cardio126phen.xlsx' %MyPath
Cardio126_AgeGenderBMIcreatSmoking=pd.read_excel(f2).set_index('BD')
print len(Cardio126_AgeGenderBMIcreatSmoking)
Cardio126_AgeGenderBMIcreatSmoking.head()

## merge phenotype files and add dummy variables:

In [ ]:
# combined phenotype table:
PNP530Cardio126_phen=pd.concat([PNP530_AgeGenderBMIcreatSmoking,Cardio126_AgeGenderBMIcreatSmoking])
PNP530Cardio126_phen=PNP530Cardio126_phen.drop(['RegistrationCode','DM','Diagnosis'], axis=1)
print PNP530Cardio126_phen.shape
print PNP530_AgeGenderBMIcreatSmoking.shape
print Cardio126_AgeGenderBMIcreatSmoking.shape

In [ ]:
PNP530Cardio126_phen['Female'] = PNP530Cardio126_phen.Gender.map({'Female':1, 'Male':0})

In [ ]:
smokingDum=pd.get_dummies(PNP530Cardio126_phen.Smoking, prefix='Smoking').iloc[:, 1:]
smokingDum.head()

In [ ]:
PNP530Cardio126_phen=pd.merge(PNP530Cardio126_phen,smokingDum,how='left',left_index=True, right_index=True)
PNP530Cardio126_phen.head()

In [ ]:
PNP530Cardio126_phen.PCRplate.value_counts(dropna=False)

In [ ]:
PlateDum=pd.get_dummies(PNP530Cardio126_phen.PCRplate, prefix='PCR')
PlateDum.head()

In [ ]:
PNP530Cardio126_phen=pd.merge(PNP530Cardio126_phen,PlateDum,how='left',left_index=True, right_index=True)
PNP530Cardio126_phen.head()

In [ ]:
f2='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen.xlsx' %MyPath
PNP530Cardio126_phen.to_excel(f2)

# logistic regression - seq presence/absence

## explore coefficient p-values

### load common binary TCR file:

In [ ]:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
datasetName='PNP530Cardio126'

f1='%s/sharingAnalysis/sharingMatrix_PNP530Cardio126_minNshared5_RA_onlyProductiveTrue__minSharedT5_OLtrimmed_binary' %datasetFolder
combTCRdfmin5=pd.read_pickle(f1)
print combTCRdfmin5.shape



In [ ]:
list(combTCRdfmin5.index)

### define X

In [ ]:
f2='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen.xlsx' %MyPath
PNP530Cardio126_phen=pd.read_excel(f2)
PNP530Cardio126_phen.head()

In [ ]:
# Xtypes:
allNum=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes',
       'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
       'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
       'PCR_Plate8', 'PCR_Plate9']
small=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'Female']
smallNoCardio=['BD', 'Age', 'BMI', 'Creatinine', 'Female']
allNumNoPlate=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes']

In [ ]:
#DEFINE WHICH FEATURES TO USE:

X=PNP530Cardio126_phen[allNum].set_index('BD')

In [ ]:
#drop all samples that don't have all feature info. check how many samples were lost
print 'X shape before na removal=%s_%s' %(X.shape[0],X.shape[1])
X=X.dropna(how='any')
print 'X shape before na removal=%s_%s' %(X.shape[0],X.shape[1])
xsamples=X.index.astype(str).tolist() # get list of samples to use

X.head()

### define function:

In [ ]:
the function logReg_withWrappe was copied to cardioFunctions.py

### run over all sequences and get p-values

In [ ]:
print len(combTCRdfmin5.columns.values)

the function was run over all sequences using eclipse (paralleled in the cluster, 500 seqs/job)

### get results:

In [ ]:
allNum=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes',
       'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
       'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
       'PCR_Plate8', 'PCR_Plate9']
small=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'Female','Smoking_Past', 'Smoking_Yes']
smallNoCardio=['BD', 'Age', 'BMI', 'Creatinine', 'Female']
allNumNoPlate=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes']

Xcols=small
Xname='small'

resultFolder='%s/TCR_real_data/PNP530Cardio126Combined/seqPredictions/%s/DFs' %(MyPath, Xname)
filenames = [f for f in listdir(resultFolder) if isfile(join(resultFolder, f))]
dfs=[f for f in filenames if 'resultDF' in f]
print len(dfs)
print dfs[:5]

df_list=[]
for file_name in dfs:
    with open('%s/%s' %(resultFolder, file_name), 'rb') as f:
        current_df=pd.read_excel(f)
    f.close()
    df_list.append(current_df)
print ('the length of df list is %s' %len(df_list))

logRegResults=pd.concat(df_list)
logRegResults.head()

In [ ]:
# #define p-values columns

# p_value_columns=['Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes',
#        'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
#        'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
#        'PCR_Plate8', 'PCR_Plate9']

p_value_columns=[x for x in Xcols if x!='BD']
print p_value_columns


In [ ]:
# add corrected p-values:
nTests=len(logRegResults)
logRegResults=logRegResults.sort_values(by='isCardio')
logRegResults['minP']=logRegResults[p_value_columns].min(axis=1)
logRegResults=add_corrected_pValues(logRegResults,pValueColumn='isCardio',nTests=nTests,FDR=0.1)
logRegResults

In [ ]:
# add column indicate sequences in which the 'isCardio' is the best p-value
logRegResults['isCardioisBest']=np.where(logRegResults['isCardio']==logRegResults[p_value_columns].min(axis=1),1,0)
logRegResults

In [ ]:
#save file:
f1='%s/TCR_real_data/PNP530Cardio126Combined/seqPredictions/resultDFlogReg_%s_TCRbinaryMin5.xlsx' %(MyPath,Xname)
logRegResults.to_excel(f1)

In [ ]:
#get descriptive statistics and save:
logRegResultsStats=logRegResults.describe()
logRegResultsStats
f1='%s/TCR_real_data/PNP530Cardio126Combined/seqPredictions/logRegResultsStats_%s_TCRbinaryMin5.xlsx' %(MyPath,Xname)
logRegResultsStats.to_excel(f1)

In [ ]:
#calculate how many times the 'is Cardio' was significnat with bonferroni correction

resultLen=len(logRegResults)
print 'result list length is %s' %resultLen
resultsStatSum=pd.DataFrame(logRegResults[p_value_columns].mean())
resultsStatSum=resultsStatSum.rename(columns={0:'mean_p_value'})
resultsStatSum['nSigBonf']=logRegResults[p_value_columns].apply(lambda x: len([n for n in x if n<(0.05/resultLen)]))
resultsStatSum['nBest']=logRegResults[p_value_columns].idxmin(axis=1).value_counts()
print resultsStatSum.sort_values(by='nSigBonf',ascending=False)

f1='%s/TCR_real_data/PNP530Cardio126Combined/seqPredictions/resultsStatSum_%s_TCRbinaryMin5.xlsx' %(MyPath,Xname)
resultsStatSum.to_excel(f1)

### plot r2 distribution

In [ ]:
prsquared=pd.to_numeric(logRegResults.prsquared,errors='coerce')
prsquared=prsquared.replace([np.inf, -np.inf], np.nan)
prsquared.sort_values(ascending=False)
prsquared=prsquared[prsquared.notnull()]
print prsquared.max()
print prsquared.min()
print len(prsquared[prsquared<0])
print float(len(prsquared[prsquared<0]))/len(prsquared)
plt.hist(prsquared[prsquared>0], bins=20)
plt.show()

In [ ]:
#compare r2 distribution in isCardioBest to the rest:
logRegResults['prsquared']=logRegResults['prsquared'].replace([np.inf, -np.inf], np.nan)
logRegResults=logRegResults[logRegResults['prsquared'].notnull()]
logRegResults=logRegResults[logRegResults['prsquared']>0]
data={}
for name, group in logRegResults.groupby('isCardioisBest'):
    data[name] =  list(group['prsquared'])
data0=data[0]
data1=data[1]
print len(data0)
print len(data1)
print len(data0)+len(data1)
s_t, p_t = stats.ttest_ind(data[0], data[1])
s_k, p_k = stats.ks_2samp(data[0], data[1])
print 'mean for group 0=%s' %np.mean(data[0])
print 'mean for group 1=%s' %np.mean(data[1])
print 'p_value ttest= %s' %round(p_t,10)
print 'p_value kstest= %s' %round(p_k,10)
data0weights=np.ones_like(data0) / len(data0)
data1weights=np.ones_like(data1) / len(data1)
plt.hist([data0,data1],bins=20,weights=[data0weights,data1weights],label=['isCardio not the best','isCardio the best'])
plt.legend()
plt.show()

# linear regression - TCR feature prediction

### generate common feature file:

In [ ]:
#load PNP530:
f1='%s/TCR_real_data/featureSummaryDFs/PNP530_allFeatures' %MyPath
PNP530_allFeatures=pd.read_pickle(f1)
print PNP530_allFeatures.shape
# print PNP530_allFeatures.head()

In [ ]:
#load Cardio:
f1='%s/TCR_real_data/CardioSamples/featureSummaryDFs/Cardio126_allFeatures' %MyPath
Cardio126_allFeatures=pd.read_pickle(f1)
print Cardio126_allFeatures.shape
# print Cardio126_allFeatures.head()

In [ ]:
PNP530Cardio126_allFeatures=pd.concat([PNP530_allFeatures,Cardio126_allFeatures])
print PNP530Cardio126_allFeatures.shape
# print PNP530Cardio126_allFeatures.head()

In [ ]:
# makedirs('%s/TCR_real_data/PNP530Cardio126Combined/featureSummaryDFs' %MyPath)

f1='%s/TCR_real_data/PNP530Cardio126Combined/featureSummaryDFs/PNP530Cardio126_allFeatures' %MyPath
PNP530Cardio126_allFeatures.to_pickle(f1)

f2='%s/TCR_real_data/PNP530Cardio126Combined/featureSummaryDFs/PNP530Cardio126_allFeatures.xlsx' %MyPath
PNP530Cardio126_allFeatures.to_excel(f2)

### change function:

In [ ]:
the function linReg_withWrapper was copied to  cardioFunctions.py

In [ ]:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
datasetName='PNP530Cardio126'

f1='%s/TCR_real_data/PNP530Cardio126Combined/featureSummaryDFs/PNP530Cardio126_allFeatures' %MyPath
PNP530Cardio126_allFeatures=pd.read_pickle(f1)
print PNP530Cardio126_allFeatures.shape



f2='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen.xlsx' %MyPath
PNP530Cardio126_phen=pd.read_excel(f2)
PNP530Cardio126_phen.head()

# Xtypes:
allNum=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes',
       'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
       'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
       'PCR_Plate8', 'PCR_Plate9']
small=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'Female','Smoking_Past', 'Smoking_Yes']
smallNoCardio=['BD', 'Age', 'BMI', 'Creatinine', 'Female']
allNumNoPlate=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes']

#DEFINE WHICH FEATURES TO USE:

Xcols=small
Xname='small'

X=PNP530Cardio126_phen[Xcols].set_index('BD')
#drop all samples that don't have all feature info. check how many samples were lost
print 'X shape before na removal=%s_%s' %(X.shape[0],X.shape[1])
X=X.dropna(how='any')
print 'X shape before na removal=%s_%s' %(X.shape[0],X.shape[1])
xsamples=X.index.astype(str).tolist() # get list of samples to use

modelType='ols'
yDF=PNP530Cardio126_allFeatures
yname='PNP530Cardio126_allFeatures'
resultFolder='%s/TCR_real_data/PNP530Cardio126Combined/FeaturePredictions/%s/%s_%s_DFs' %(MyPath,modelType,Xname,yname)
if not isdir(resultFolder):
    makedirs(resultFolder)
yMinCol=None
yMaxCol=None
maxiter=1000


df=linearOrlogistic_reg_withWrapper(X,Xname,yDF,yname,resultFolder,modelType,yMinCol,yMaxCol,maxiter)

### get results:

In [ ]:
f1='%s/resultDF_regType%s_%s_%s_ycol0_ycol746_mi%s.xlsx'   %(resultFolder,modelType,Xname,yname,maxiter)
linRegResults=pd.read_excel(f1)

In [ ]:
#define p-values columns
p_value_columns=[x for x in Xcols if x!='BD']
print p_value_columns

In [ ]:
# add corrected p-values:
nTests=len(linRegResults)
linRegResults=linRegResults.sort_values(by='isCardio')
# linRegResults['minP']=linRegResults[p_value_columns].min(axis=1)
linRegResults=add_corrected_pValues(linRegResults,pValueColumn='isCardio',nTests=nTests,FDR=0.1)
linRegResults

In [ ]:
# add column indicate sequences in which the 'isCardio' is the best p-value
linRegResults['isCardioisBest']=np.where(linRegResults['isCardio']==linRegResults[p_value_columns].min(axis=1),1,0)
linRegResults

In [ ]:
#save file:
f1='%s/resultDF_regType%s_%s_%s_ycol0_ycol746_mi%s.xlsx'   %(resultFolder,modelType,Xname,yname,maxiter)
linRegResults.to_excel(f1)

In [ ]:
#get descriptive statistics and save:
linRegResultsStats=linRegResults.describe()
linRegResultsStats
f1='%s/linRegResultsStats_%s_%s_ycol0_ycol746_mi%s.xlsx'   %(resultFolder,Xname,yname,maxiter)
linRegResultsStats.to_excel(f1)

In [ ]:
#calculate how many times the 'is Cardio' was significnat with bonferroni correction

resultLen=len(linRegResults)
print 'result list length is %s' %resultLen
resultsStatSum=pd.DataFrame(linRegResults[p_value_columns].mean())
resultsStatSum=resultsStatSum.rename(columns={0:'mean_p_value'})
resultsStatSum['nSigBonf']=linRegResults[p_value_columns].apply(lambda x: len([n for n in x if n<(0.05/resultLen)]))
resultsStatSum['nBest']=linRegResults[p_value_columns].idxmin(axis=1).value_counts()
print resultsStatSum.sort_values(by='nSigBonf',ascending=False)
f1='%s/linRegResultsStatSum_%s_%s_ycol0_ycol746_mi%s.xlsx'   %(resultFolder,Xname,yname,maxiter)
resultsStatSum.to_excel(f1)

In [ ]:
resultsStatSum

### plot r2 distribution

In [ ]:
rsquared_adj=pd.to_numeric(linRegResults.rsquared_adj,errors='coerce')
rsquared_adj=rsquared_adj.replace([np.inf, -np.inf], np.nan)
rsquared_adj.sort_values(ascending=False)
rsquared_adj=rsquared_adj[rsquared_adj.notnull()]
print rsquared_adj.max()
print rsquared_adj.min()
plt.hist(rsquared_adj, bins=20)
plt.show()

# logistic regression - predict healthy/sick using TCR seq or features

In [ ]:
#the function  predictBinaryPhenotype_logReg_TCRfeatures was copied to CardioFunctions.py

## run for all PNP530cARDIO126 samples - TCR features

In [ ]:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
datasetName='PNP530Cardio126'

#get feature df:
f1='%s/TCR_real_data/PNP530Cardio126Combined/featureSummaryDFs/PNP530Cardio126_allFeatures' %MyPath
PNP530Cardio126_allFeatures=pd.read_pickle(f1)
print PNP530Cardio126_allFeatures.shape

#get phenotype df:

## full phenotype df:
f2='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen.xlsx' %MyPath
PNP530Cardio126_phen=pd.read_excel(f2)
PNP530Cardio126_phen.head()

###

# Xtypes:
allNum=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes',
       'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
       'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
       'PCR_Plate8', 'PCR_Plate9']
small=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'Female','Smoking_Past', 'Smoking_Yes']
smallNoCardio=['BD', 'Age', 'BMI', 'Creatinine', 'Female']
allNumNoPlate=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes']

#DEFINE WHICH FEATURES TO USE:

Xcols=small
phenotypeDFname='small'
Xname=phenotypeDFname
phenotypeDF=PNP530Cardio126_phen[Xcols].set_index('BD')
TCRvariableDF=PNP530Cardio126_allFeatures
TCRvariableName='PNP530Cardio126_allFeatures'
targetDF=phenotypeDF['isCardio']
phenotypeDF=phenotypeDF.drop('isCardio',axis=1)
targetName='isCardio'
modelType='logit'
# print 'target original:'
# print targetDF.head(10)
# print targetDF.value_counts(dropna=False)



resultFolder='%s/TCR_real_data/PNP530Cardio126Combined/%s_Predictions/%s/%s_%s_DFs' %(MyPath,targetName,modelType,
                                                                                      phenotypeDFname,TCRvariableName)
if not isdir(resultFolder):
    makedirs(resultFolder)
    
    
TCRdfMinCol=None
TCRdfMaxCol=None
maxiter=1000



df=predictBinaryPhenotype_logReg_TCRfeatures(phenotypeDF, phenotypeDFname, TCRvariableDF, TCRvariableName, targetDF, 
                                               targetName, resultFolder, modelType, TCRdfMinCol, TCRdfMaxCol, 
                                               maxiter)

In [ ]:
df.head()

### get results:

In [ ]:
f1 = '%s/resultDF_regType%s_%s_%s_ycol0_ycol746_mi%s.xlsx' % (resultFolder, modelType, phenotypeDFname, TCRvariableName, 
                                                               maxiter)
logRegResults=pd.read_excel(f1)
logRegResults.head()

In [ ]:
# #define p-values columns

p_value_columns=[x for x in Xcols if (x!='BD') and (x!='isCardio')]
p_value_columns=p_value_columns+['variable_p']
print p_value_columns


In [ ]:
# add corrected p-values:
nTests=len(logRegResults)
logRegResults=logRegResults.sort_values(by='variable_p')
logRegResults['minP']=logRegResults[p_value_columns].min(axis=1)
logRegResults=add_corrected_pValues(logRegResults,pValueColumn='variable_p',nTests=nTests,FDR=0.1)
logRegResults

In [ ]:
# add column indicate sequences in which the 'isCardio' is the best p-value
logRegResults['isVariableBest']=np.where(logRegResults['variable_p']==logRegResults[p_value_columns].min(axis=1),1,0)
logRegResults

In [ ]:
#save file:
f1 = '%s/resultDF_regType%s_%s_%s_ycol0_ycol746_mi%s_processed.xlsx' % (resultFolder, modelType, phenotypeDFname, TCRvariableName, 
                                                               maxiter)
logRegResults.to_excel(f1)

In [ ]:
logRegResults.columns.values

In [ ]:
#get descriptive statistics and save:
Xname=phenotypeDFname
logRegResultsStats=logRegResults.describe()
logRegResultsStats
f1='%s/logRegResultsStats_%s_%s.xlsx' %(resultFolder,Xname,TCRvariableName)
logRegResultsStats.to_excel(f1)

In [ ]:
#calculate how many times the 'is Cardio' was significnat with bonferroni correction

resultLen=len(logRegResults)
print 'result list length is %s' %resultLen
resultsStatSum=pd.DataFrame(logRegResults[p_value_columns].mean())
resultsStatSum=resultsStatSum.rename(columns={0:'mean_p_value'})
resultsStatSum['nSigBonf']=logRegResults[p_value_columns].apply(lambda x: len([n for n in x if n<(0.05/resultLen)]))
resultsStatSum['nBest']=logRegResults[p_value_columns].idxmin(axis=1).value_counts()
print resultsStatSum.sort_values(by='nSigBonf',ascending=False)

f1='%s/resultsStatSum_%s_%s.xlsx' %(resultFolder,Xname,TCRvariableName)
resultsStatSum.to_excel(f1)

### plot r2 distribution

In [ ]:
adj_prsquared=pd.to_numeric(logRegResults.adj_prsquared,errors='coerce')
adj_prsquared=adj_prsquared.replace([np.inf, -np.inf], np.nan)
adj_prsquared.sort_values(ascending=False)
adj_prsquared=adj_prsquared[adj_prsquared.notnull()]
print adj_prsquared.max()
print adj_prsquared.min()
print len(adj_prsquared[adj_prsquared<0])
print float(len(adj_prsquared[adj_prsquared<0]))/len(adj_prsquared)
plt.hist(adj_prsquared[adj_prsquared>0], bins=20)
plt.show()

In [ ]:
logRegResults['isVariableBest'].value_counts()

In [ ]:
#compare r2 distribution in isCardioBest to the rest:
logRegResults['adj_prsquared']=logRegResults['adj_prsquared'].replace([np.inf, -np.inf], np.nan)
logRegResults=logRegResults[logRegResults['adj_prsquared'].notnull()]
logRegResults=logRegResults[logRegResults['adj_prsquared']>0]
data={}
for name, group in logRegResults.groupby('isVariableBest'):
    data[name] =  list(group['adj_prsquared'])
data0=data[0]
data1=data[1]
print len(data0)
print len(data1)
print len(data0)+len(data1)
s_t, p_t = stats.ttest_ind(data[0], data[1])
s_k, p_k = stats.ks_2samp(data[0], data[1])
print 'mean for group 0=%s' %np.mean(data[0])
print 'mean for group 1=%s' %np.mean(data[1])
print 'p_value ttest= %s' %round(p_t,10)
print 'p_value kstest= %s' %round(p_k,10)
data0weights=np.ones_like(data0) / len(data0)
data1weights=np.ones_like(data1) / len(data1)
plt.hist([data0,data1],bins=20,weights=[data0weights,data1weights],label=['isVariable not the best','isVariable the best'])
plt.legend()
plt.show()

### compare result to feature selection option:

#### univariate feature selection:

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,mutual_info_classif,f_classif


In [ ]:
nNans=50

In [ ]:
X=pd.merge(phenotypeDF,TCRvariableDF,how='outer',left_index=True,right_index=True)
print X.shape
nullSum=X.isnull().sum()
print nullSum.sort_values(ascending=False).head()
print len(nullSum[nullSum>nNans])


In [ ]:
ignoreCollumns=nullSum[nullSum>nNans].index.tolist()
print ignoreCollumns[:5]

In [ ]:
X=X.drop(ignoreCollumns,axis=1)

In [ ]:
print X.shape
X=X.dropna(how='any')
print X.shape

In [ ]:
targetName='isCardio'
y=targetDF
y=y.rename(targetName)
      
xsamples = X.index.astype(str).tolist()  # get list of samples to use
y = y.dropna(how='any')
y = y.loc[xsamples]

In [ ]:
print X.shape
print y.shape


In [ ]:
f_classif=mutual_info_classif
k=20

In [ ]:
X_new = SelectKBest(f_classif, k).fit_transform(X, y)
X_new.shape


In [ ]:
selector = SelectKBest(f_classif, k)
selector.fit(X,y)

idxs_selected = selector.get_support(indices=True)
# Create new dataframe with only desired columns, or overwrite existing
X_new = X.iloc[:,idxs_selected]

In [ ]:
print X_new.columns.values

In [ ]:
print pd.DataFrame(X_new).columns.values

## run for the matched dataset (86) - TCR features

In [ ]:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126Matched_ss9000'

#get sample list:
folder='%s/SamplesForAnalysis_corrected' %datasetFolder
matched86List=gen_sampleList_from_Folder(folder,'PNP530Cardio126Matched86')

#get feature df:
f1='%s/TCR_real_data/PNP530Cardio126Combined/featureSummaryDFs/PNP530Cardio126_allFeatures' %MyPath
PNP530Cardio126_allFeatures=pd.read_pickle(f1)
print PNP530Cardio126_allFeatures.shape
PNP530Cardio126_allFeatures=editSampleNames(PNP530Cardio126_allFeatures)
PNP530Cardio126matched86_allFeatures=PNP530Cardio126_allFeatures.loc[matched86List,:]
print PNP530Cardio126matched86_allFeatures.shape

#get phenotype df:

## full phenotype df:
f2='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen.xlsx' %MyPath
PNP530Cardio126_phen=pd.read_excel(f2).set_index('BD')
PNP530Cardio126_phen.head()
PNP530Cardio126_phen=editSampleNames(PNP530Cardio126_phen)
PNP530Cardio126matched86_phen=PNP530Cardio126_phen.loc[matched86List,:]
print PNP530Cardio126matched86_phen.shape



###

# Xtypes:
allNum=['Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes',
       'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
       'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
       'PCR_Plate8', 'PCR_Plate9']
small=['Age', 'BMI', 'Creatinine', 'isCardio', 'Female','Smoking_Past', 'Smoking_Yes']
smallNoCardio=['Age', 'BMI', 'Creatinine', 'Female']
allNumNoPlate=['Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes']

#DEFINE WHICH FEATURES TO USE:

Xcols=small
phenotypeDFname='small'
Xname=phenotypeDFname
phenotypeDF=PNP530Cardio126matched86_phen[Xcols]
TCRvariableDF=PNP530Cardio126matched86_allFeatures
TCRvariableName='PNP530Cardio126matched86_allFeatures'
targetDF=phenotypeDF['isCardio']
phenotypeDF=phenotypeDF.drop('isCardio',axis=1)
targetName='isCardio'
modelType='logit'
# print 'target original:'
# print targetDF.head(10)
# print targetDF.value_counts(dropna=False)



resultFolder='%s/%s_Predictions/%s/%s_%s_DFs' %(datasetFolder,targetName,modelType,phenotypeDFname,TCRvariableName)
if not isdir(resultFolder):
    makedirs(resultFolder)
    
    
TCRdfMinCol=None
TCRdfMaxCol=None
maxiter=1000



df=predictBinaryPhenotype_logReg_TCRfeatures(phenotypeDF, phenotypeDFname, TCRvariableDF, TCRvariableName, targetDF, 
                                               targetName, resultFolder, modelType, TCRdfMinCol, TCRdfMaxCol, 
                                               maxiter)

In [ ]:
df.head()

### get results:

In [ ]:
f1 = '%s/resultDF_regType%s_%s_%s_ycol0_ycol746_mi%s.xlsx' % (resultFolder, modelType, phenotypeDFname, TCRvariableName, 
                                                               maxiter)
logRegResults=pd.read_excel(f1)
logRegResults.head()

In [ ]:
# #define p-values columns

p_value_columns=[x for x in Xcols if (x!='BD') and (x!='isCardio')]
p_value_columns=p_value_columns+['variable_p']
print p_value_columns


In [ ]:
# add corrected p-values:
nTests=len(logRegResults)
logRegResults=logRegResults.sort_values(by='variable_p')
logRegResults['minP']=logRegResults[p_value_columns].min(axis=1)
logRegResults=add_corrected_pValues(logRegResults,pValueColumn='variable_p',nTests=nTests,FDR=0.1)
logRegResults.head()

In [ ]:
# add column indicate sequences in which the 'isCardio' is the best p-value
logRegResults['isVariableBest']=np.where(logRegResults['variable_p']==logRegResults[p_value_columns].min(axis=1),1,0)
logRegResults.head()

In [ ]:
#save file:
f1 = '%s/resultDF_regType%s_%s_%s_ycol0_ycol746_mi%s_processed.xlsx' % (resultFolder, modelType, phenotypeDFname, TCRvariableName, 
                                                               maxiter)
logRegResults.to_excel(f1)

In [ ]:
logRegResults.columns.values

In [ ]:
#get descriptive statistics and save:
Xname=phenotypeDFname
logRegResultsStats=logRegResults.describe()
logRegResultsStats
f1='%s/logRegResultsStats_%s_%s.xlsx' %(resultFolder,Xname,TCRvariableName)
logRegResultsStats.to_excel(f1)

In [ ]:
#calculate how many times the 'is Cardio' was significnat with bonferroni correction

resultLen=len(logRegResults)
print 'result list length is %s' %resultLen
resultsStatSum=pd.DataFrame(logRegResults[p_value_columns].mean())
resultsStatSum=resultsStatSum.rename(columns={0:'mean_p_value'})
resultsStatSum['nSigBonf']=logRegResults[p_value_columns].apply(lambda x: len([n for n in x if n<(0.05/resultLen)]))
resultsStatSum['nBest']=logRegResults[p_value_columns].idxmin(axis=1).value_counts()
print resultsStatSum.sort_values(by='nSigBonf',ascending=False)

f1='%s/resultsStatSum_%s_%s.xlsx' %(resultFolder,Xname,TCRvariableName)
resultsStatSum.to_excel(f1)

## run for the all PNP530Cardio samples - seq pres/abs

ran on eclipse using parallel jobs:

### get results:

In [ ]:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
datasetName='PNP530Cardio126'

f1='%s/sharingAnalysis/sharingMatrix_PNP530Cardio126_minNshared5_RA_onlyProductiveTrue__minSharedT5_OLtrimmed_binary' %datasetFolder
combTCRdfmin5=pd.read_pickle(f1)
print combTCRdfmin5.shape

#get phenotype df:
f2='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen.xlsx' %MyPath
PNP530Cardio126_phen=pd.read_excel(f2)
PNP530Cardio126_phen.head()

# Xtypes:
allNum=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes',
       'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
       'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
       'PCR_Plate8', 'PCR_Plate9']
small=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'Female','Smoking_Past', 'Smoking_Yes']
smallNoCardio=['BD', 'Age', 'BMI', 'Creatinine', 'Female']
allNumNoPlate=['BD', 'Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Female', 'Smoking_Past', 'Smoking_Yes']

#DEFINE WHICH FEATURES TO USE:

Xcols=small
phenotypeDFname='small'
Xname=phenotypeDFname
phenotypeDF=PNP530Cardio126_phen[Xcols].set_index('BD')
TCRvariableDF=combTCRdfmin5
TCRvariableName='combTCRdfmin5'
targetDF=phenotypeDF['isCardio']
phenotypeDF=phenotypeDF.drop('isCardio',axis=1)
targetName='isCardio'
modelType='logit'
# print 'target original:'
# print targetDF.head(10)
# print targetDF.value_counts(dropna=False)



resultFolder='%s/TCR_real_data/PNP530Cardio126Combined/%s_Predictions/%s/%s_%s_DFs' %(MyPath,targetName,modelType,
                                                                                      phenotypeDFname,TCRvariableName)


In [ ]:
filenames = [f for f in listdir(resultFolder) if isfile(join(resultFolder, f))]
dfs=[f for f in filenames if 'resultDF' in f]
print len(dfs)
print dfs[:5]

df_list=[]
for file_name in dfs:
    with open('%s/%s' %(resultFolder, file_name), 'rb') as f:
        current_df=pd.read_excel(f)
    f.close()
    df_list.append(current_df)
print ('the length of df list is %s' %len(df_list))

logRegResults=pd.concat(df_list)
logRegResults.head()

In [ ]:
# #define p-values columns

p_value_columns=[x for x in Xcols if (x!='BD') and (x!='isCardio')]
p_value_columns=p_value_columns+['variable_p']
print p_value_columns


In [ ]:
# add corrected p-values:
nTests=len(logRegResults)
logRegResults=logRegResults.sort_values(by='variable_p')
logRegResults['minP']=logRegResults[p_value_columns].min(axis=1)
logRegResults=add_corrected_pValues(logRegResults,pValueColumn='variable_p',nTests=nTests,FDR=0.1)
logRegResults.head()

In [ ]:
# add column indicate sequences in which the 'isCardio' is the best p-value
logRegResults['isVariableBest']=np.where(logRegResults['variable_p']==logRegResults[p_value_columns].min(axis=1),1,0)
logRegResults.head()

In [ ]:
#save file:
maxiter=1000
f1 = '%s/resultDF_regType%s_%s_%s_ycol0_ycol746_mi%s_processed.xlsx' % (resultFolder, modelType, phenotypeDFname, TCRvariableName, 
                                                               maxiter)
logRegResults.to_excel(f1)

In [ ]:
logRegResults.columns.values

In [ ]:
#get descriptive statistics and save:
Xname=phenotypeDFname
logRegResultsStats=logRegResults.describe()
logRegResultsStats
f1='%s/logRegResultsStats_%s_%s.xlsx' %(resultFolder,Xname,TCRvariableName)
logRegResultsStats.to_excel(f1)

In [ ]:
#calculate how many times the 'is Cardio' was significnat with bonferroni correction

resultLen=len(logRegResults)
print 'result list length is %s' %resultLen
resultsStatSum=pd.DataFrame(logRegResults[p_value_columns].mean())
resultsStatSum=resultsStatSum.rename(columns={0:'mean_p_value'})
resultsStatSum['nSigBonf']=logRegResults[p_value_columns].apply(lambda x: len([n for n in x if n<(0.05/resultLen)]))
resultsStatSum['nBest']=logRegResults[p_value_columns].idxmin(axis=1).value_counts()
print resultsStatSum.sort_values(by='nSigBonf',ascending=False)

f1='%s/resultsStatSum_%s_%s.xlsx' %(resultFolder,Xname,TCRvariableName)
resultsStatSum.to_excel(f1)

### plot r2 distribution

In [ ]:
adj_prsquared=pd.to_numeric(logRegResults.adj_prsquared,errors='coerce')
adj_prsquared=adj_prsquared.replace([np.inf, -np.inf], np.nan)
adj_prsquared.sort_values(ascending=False)
adj_prsquared=adj_prsquared[adj_prsquared.notnull()]
print adj_prsquared.max()
print adj_prsquared.min()
print len(adj_prsquared[adj_prsquared<0])
print float(len(adj_prsquared[adj_prsquared<0]))/len(adj_prsquared)
plt.hist(adj_prsquared[adj_prsquared>0], bins=20)
plt.show()

In [ ]:
logRegResults['isVariableBest'].value_counts()

In [ ]:
#compare r2 distribution in isCardioBest to the rest:
logRegResults['adj_prsquared']=logRegResults['adj_prsquared'].replace([np.inf, -np.inf], np.nan)
logRegResults=logRegResults[logRegResults['adj_prsquared'].notnull()]
logRegResults=logRegResults[logRegResults['adj_prsquared']>0]
data={}
for name, group in logRegResults.groupby('isVariableBest'):
    data[name] =  list(group['adj_prsquared'])
data0=data[0]
data1=data[1]
print len(data0)
print len(data1)
print len(data0)+len(data1)
s_t, p_t = stats.ttest_ind(data[0], data[1])
s_k, p_k = stats.ks_2samp(data[0], data[1])
print 'mean for group 0=%s' %np.mean(data[0])
print 'mean for group 1=%s' %np.mean(data[1])
print 'p_value ttest= %s' %round(p_t,10)
print 'p_value kstest= %s' %round(p_k,10)
data0weights=np.ones_like(data0) / len(data0)
data1weights=np.ones_like(data1) / len(data1)
plt.hist([data0,data1],bins=20,weights=[data0weights,data1weights],label=['isVariable not the best','isVariable the best'])
plt.legend()
plt.show()